In [1]:
import json
import pandas as pd
import numpy as np
import random
from bidict import bidict
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
from collections import Counter
import re

In [88]:
%load_ext cythonmagic

/u/solimanz/development/career_prediction_gpu/lib/python3.6/site-packages/IPython/extensions/cythonmagic.py:21: UserWarning: The Cython magic has been moved to the Cython package
  warnings.warn("""The Cython magic has been moved to the Cython package""")


In [2]:
%matplotlib inline

In [3]:
sns.set(style="darkgrid", color_codes=True)
plt.style.use('ggplot')

In [4]:
raw_df = pd.read_pickle('/data/rali7/Tmp/solimanz/data/pickles/clean.pkl')

In [5]:
def fmtcols(mylist, cols):
    maxwidth = max(map(lambda x: len(x), mylist))
    justifyList = list(map(lambda x: x.ljust(maxwidth), mylist))
    lines = (' '.join(justifyList[i:i+cols]) for i in range(0,len(justifyList),cols))
    return "\n".join(lines)

In [6]:
def apply_transforms(s):
    for transform in transforms:
        s = re.sub(*transform, s)
    return s

In [86]:
transforms = [
    # senior/junior
    (re.compile(r'\bsr\.(?!\s)'), 'senior '),
    (re.compile(r'\bjr\.(?!\s)'), 'junior '),
    (re.compile(r'\bsr\.(?!\S)'), 'senior'),
    (re.compile(r'\bjr\.(?!\S)'), 'junior'),
    
    (re.compile(r'\bsenior\.(?!\S)'), 'senior'),
    (re.compile(r'\bjunior\.(?!\S)'), 'junior'),
    (re.compile(r'\bsenior\.(?!\s)'), 'senior '),
    (re.compile(r'\bjunior\.(?!\s)'), 'junior '),
    
    (re.compile(r'\bsr\b'), 'senior'),
    (re.compile(r'\bjr\b'), 'junior'),
    
    (re.compile(r'\bi\.*t\.*(?!\s)'), 'information technology '),
    (re.compile(r'\bi\.*t\.*(?!\S)'), 'information technology'),
    # C*O
    (re.compile(r'\bc\.*e\.*o\.*(?!\S)'), 'chief executive officer'),
    (re.compile(r'\bc\.*o\.*o\.*(?!\S)'), 'chief operating officer'),
    (re.compile(r'\bc\.*t\.*o\.*(?!\S)'), 'chief technology officer'),
    (re.compile(r'\bc\.*f\.*o\.*(?!\S)'), 'chief finance officer'),
    (re.compile(r'\bchief financial officer\b'), 'chief finance officer'),
    (re.compile(r'\bchief operations officer\b'), 'chief operating officer'),
    # VP
    (re.compile(r'\bv\.*p\.*(?!\s)'), 'vice president '),
    (re.compile(r'\bv\.*p\.*(?!\S)'), 'vice president'),
    #technician vs tech
    (re.compile(r'\btech\.*$'), 'technician'),
    # cofounder
    (re.compile(r'\bco[-|\s]founder\b'), 'cofounder'), 
    #coop
    (re.compile(r'\bco[-|\s]op\b'), 'coop'),
    #addon
    (re.compile(r'\badd[-|\s]on\b'), 'addon'),
    
    (re.compile(r'\br\.n\.'), 'registered nurse'),
    (re.compile(r'\br\.n\.\b'), 'registered nurse '),
    (re.compile(r'\brn\b'), 'registered nurse'),
    
    (re.compile(r'\br\.p\.n\.'), 'registered practical nurse'),
    (re.compile(r'\br\.p\.n\.\b'), 'registered practical nurse '),
    (re.compile(r'\brpn\b'), 'registered practical nurse'),
    (re.compile(r'\bfreelance\b'), 'freelancer'),
    # HR
    (re.compile(r'\bhr\b'), 'human resources'),
    (re.compile(r'\bh\.r\.\b'), 'human resources '),
    (re.compile(r'\bh\.r\.'), 'human resources'),
    
    (re.compile(r'\bcsr\b'), 'customer service representative'),
    (re.compile(r'\bcustomer service rep\b'), 'customer service representative'),
    (re.compile(r'\bcustomer service rep\.'), 'customer service representative'),
    (re.compile(r'\bc\.s\.r\.'), 'customer service representative'),
    
    # qa / qc
    (re.compile(r'\bq\.a\.\b'), 'quality assurance '),
    (re.compile(r'\bq\.a\.'), 'quality assurance'),
    (re.compile(r'\bq\.c\.\b'), 'quality control '),
    (re.compile(r'\bq\.c\.'), 'quality control'),
    (re.compile(r'\bdesiginer\b'), 'designer'),
    (re.compile(r'\bbiomed\b'), 'biomedical'),
    (re.compile(r'\bgoverenment\b'), 'government'),
    (re.compile(r'\bmachanic\b'), 'mechanic'),
    (re.compile(r'business owner'), 'owner'),
    (re.compile(r'\br[\s]*&[\s]*d\b'), 'research/development'),
    (re.compile(r'\br and d\b'), 'research/development'),
    (re.compile(r'[\s]*&[\s]*'), '/'),
    (re.compile(r'[\s]*,[\s]*'), '/'),
    (re.compile(r'[\s]*/[\s]*'), '/'),
    (re.compile(r'[\s]*\|[\s]*'), '/'),
    (re.compile(r'[\s]+and[\s]+'), '/'),
    (re.compile(r'\beditor-in-chief\b'), 'editor in chief'),
    #(re.compile(r'[\s]*-[\s]*'), '/'),
    # Remove all parens and there content
    (re.compile(r'\([\w\s\W\S]*\)'), ''),
    
    #ESL
    (re.compile(r'^esl$'), 'english as a second language instructor'),
    (re.compile(r'^esl instructor$'), 'english as a second language instructor'),
    (re.compile(r'^esl teacher$'), 'english as a second language instructor'),
    (re.compile(r'^english as a second language$'), 'english as a second language instructor'),
    (re.compile(r'^english as a second language teacher$'), 'english as a second language instructor'),
    (re.compile(r'^e.s.l. instructor$'), 'english as a second language instructor'),
    (re.compile(r'^e.s.l. teacher$'), 'english as a second language instructor')
    ]

In [87]:
funcs_series = raw_df.function.apply(apply_transforms)

In [89]:
func_counts = funcs_series.value_counts()

In [90]:
with open("/data/rali7/Tmp/solimanz/data_viz/top_titles.txt", "w") as f:
    f.write(fmtcols(func_counts.index.values[:550], 3))

In [41]:
func_counts['coo']

1518

In [84]:
re.sub(re.compile(r'\bc\.*f\.*o\.*(\b)'), 'chief finance officer', 'cfo.')

'chief finance officer.'